# Fine-tune FLAN-T5 for CG Classification

## 1. Setup Development Environment


In [ ]:
!pip install pytesseract transformers==4.28.1 datasets evaluate rouge-score nltk tensorboard py7zr

# from huggingface_hub import notebook_login
# notebook_login()

### Connect to Drive

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import glob
from datasets import load_dataset
import datasets

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Load and prepare dataset

In [ ]:
import pickle

f = open("/content/drive/MyDrive/Corpus/CG_Corpus/cg_3to1_8previous_event_selection.dat", "rb")
dataset = pickle.load(f)
f.close()
dataset

DatasetDict({
    train: Dataset({
        features: ['Speaker', 'Sentence_Number', 'Sentence', 'Event', 'Target_Event', 'Bel(A)', 'Bel(B)', 'CG(A)', 'CG(B)'],
        num_rows: 970
    })
    test: Dataset({
        features: ['Speaker', 'Sentence_Number', 'Sentence', 'Event', 'Target_Event', 'Bel(A)', 'Bel(B)', 'CG(A)', 'CG(B)'],
        num_rows: 325
    })
})

### Preprocess

In [ ]:
import pandas as pd
from datasets import Dataset

train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])
train_df['Bel(A)'] = train_df['Bel(A)'].astype(str)
test_df['Bel(A)'] = test_df['Bel(A)'].astype(str)
train_df['CG(A)'] = train_df['CG(A)'].astype(str)
test_df['CG(A)'] = test_df['CG(A)'].astype(str)
train_df['Bel(B)'] = train_df['Bel(B)'].astype(str)
test_df['Bel(B)'] = test_df['Bel(B)'].astype(str)
train_df['CG(B)'] = train_df['CG(B)'].astype(str)
test_df['CG(B)'] = test_df['CG(B)'].astype(str)
dataset['train'] = Dataset.from_pandas(train_df)
dataset['test'] = Dataset.from_pandas(test_df)
dataset['train'] = dataset['train'].shuffle()
dataset

DatasetDict({
    train: Dataset({
        features: ['Speaker', 'Sentence_Number', 'Sentence', 'Event', 'Target_Event', 'Bel(A)', 'Bel(B)', 'CG(A)', 'CG(B)'],
        num_rows: 970
    })
    test: Dataset({
        features: ['Speaker', 'Sentence_Number', 'Sentence', 'Event', 'Target_Event', 'Bel(A)', 'Bel(B)', 'CG(A)', 'CG(B)'],
        num_rows: 325
    })
})

In [ ]:
train_df = dataset['train'].to_pandas()
train_df = train_df[train_df['CG(A)'] != '0']
dataset['train'] = Dataset.from_pandas(train_df)
dataset['train'] = dataset['train'].remove_columns('__index_level_0__')

test_df = dataset['test'].to_pandas()
test_df = test_df[test_df['CG(A)'] != '0']
dataset['test'] = Dataset.from_pandas(test_df)
dataset['test'] = dataset['test'].remove_columns('__index_level_0__')
dataset

DatasetDict({
    train: Dataset({
        features: ['Speaker', 'Sentence_Number', 'Sentence', 'Event', 'Target_Event', 'Bel(A)', 'Bel(B)', 'CG(A)', 'CG(B)'],
        num_rows: 889
    })
    test: Dataset({
        features: ['Speaker', 'Sentence_Number', 'Sentence', 'Event', 'Target_Event', 'Bel(A)', 'Bel(B)', 'CG(A)', 'CG(B)'],
        num_rows: 311
    })
})

In [ ]:
dataset['train'][20]

{'Speaker': 'B',
 'Sentence_Number': 20,
 'Sentence': 'B: %eh w- listen. Was it difficult coming back, going back to the &U &S? ',
 'Event': "Previous Sentences: A had a dream this morning A's dream is weird A doesn't  know if B was in A's dream B was in A's dream A thinks that B was in A's dream Barry was definitely in A's dream In A's dream, Barry and A were trying to get somewhere In A's dream, Barry and A had to jump in this water but Barry would not let go of A when Barry and A got in the water In A's dream, Barry and A had to jump in this water In A's dream, Barry would not let go of A when Barry and A got in the water In A's dream, Barry would not let go of A In A's dream, Barry and A got in the water A woke up because A started drowning in A's dream A woke up A started drowning in A's dream A doesn't know what A's dream mean A's dream means something A got a postcard from Barry too the other day A's dream is scary A's dream was pretty scary A's dream was kind of one of those fr

In [ ]:
from random import randrange

sample = dataset['train'][randrange(len(dataset["train"]))]
print(f"Input Event with Context: \n{sample['Event']}\n")
print(f"Bel(A)={sample['Bel(A)']},Bel(B)={sample['Bel(B)']}")
print(f"CG(A)={sample['CG(A)']},CG(B)={sample['CG(B)']}")

Input Event with Context: 
Previous Sentences: Kevin and A have really turned into yuppies Kevin and A have really turned into yuppies is scary B and B's partner shouldn't have a house in the suburbs B and B's partner have a house in the suburbs A asks B if A can't be a yuppie if A is unemployed A can't be a yuppie if A is unemployed A can't be a yuppie A is unemployed A asks if A needs to be a professional to be a yuppie A needs to be a professional to be a yuppie A is a yuppie B says %um. yeah I guess.   Norm and B are worse because Norm and B live in the suburbs Norm and B are worse Norm and B live in the suburbs Kevin and A will save A's impossibility of being a yuppie for Kevin's and A's self respect A asks B if B is coming to visit A and Kevin or not B is coming to visit A and Kevin B is not coming to visit A and Kevin B doesn't know if B is coming to visit A and Kevin 
Target Sentence: B's coming is up to Norm

Bel(A)=1,Bel(B)=1
CG(A)=1,CG(B)=1


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["Event"], truncation=True), batched=True, remove_columns=['Speaker', 'Sentence_Number', 'Sentence', 'Event', 'Target_Event', 'Bel(A)', 'Bel(B)', 'CG(A)', 'CG(B)'])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["CG(A)"], truncation=True), batched=True, remove_columns=['Speaker', 'Sentence_Number', 'Sentence', 'Event', 'Target_Event', 'Bel(A)', 'Bel(B)', 'CG(A)', 'CG(B)'])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Max source length: 512


Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Max target length: 2


In [ ]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["Bel(A)=" + sample['Bel(A)'][i] + ",Bel(B)=" + sample['Bel(B)'][i] + "\n\nInput Event with Context:\n" + sample['Event'][i] for i in range(len(sample["Event"]))]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["CG(A)"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['Speaker', 'Sentence_Number', 'Sentence', 'Event', 'Target_Event', 'Bel(A)', 'Bel(B)', 'CG(A)', 'CG(B)'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/889 [00:00<?, ? examples/s]

Map:   0%|          | 0/311 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


## 3. Fine-tune and evaluate FLAN-T5

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model_id="google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

We want to evaluate our model during training. The `Trainer` supports evaluation during training by providing a `compute_metrics`.
The most commonly used metrics to evaluate summarization task is [rogue_score](https://en.wikipedia.org/wiki/ROUGE_(metric)) short for Recall-Oriented Understudy for Gisting Evaluation). This metric does not behave like the standard accuracy: it will compare a generated summary against a set of reference summaries

We are going to use `evaluate` library to evaluate the `rogue` score.

In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("f1")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, average='macro')
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Before we can start training is to create a `DataCollator` that will take care of padding our inputs and labels. We will use the `DataCollatorForSeq2Seq` from the 🤗 Transformers library.

In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


The last step is to define the hyperparameters (`TrainingArguments`) we want to use for our training. We are leveraging the [Hugging Face Hub](https://huggingface.co/models) integration of the `Trainer` to automatically push our checkpoints, logs and metrics during training into a repository.

In [ ]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-event-extraction"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=3e-4,

    num_train_epochs=12,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="epoch",
    # logging_steps=300,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=False,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    # push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

We can start our training by using the `train` method of the `Trainer`.

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Gen Len
1,0.328200,0.269725,62.148000,2.000000
2,0.171000,0.198147,64.347500,2.000000
3,0.145800,0.153337,68.635900,2.000000
4,0.096300,0.189449,76.607200,2.000000
5,0.053900,0.339242,80.274100,2.000000
6,0.037800,0.333865,78.030400,2.000000
7,0.026200,0.399426,78.731200,2.000000
8,0.017700,0.443384,79.382200,2.000000
9,0.015500,0.316746,78.447400,2.000000
10,0.007300,0.418206,78.731200,2.000000


TrainOutput(global_step=1344, training_loss=0.07548848776856348, metrics={'train_runtime': 1927.4858, 'train_samples_per_second': 5.535, 'train_steps_per_second': 0.697, 'total_flos': 7304991435915264.0, 'train_loss': 0.07548848776856348, 'epoch': 12.0})

Nice, we have trained our model. 🎉 Lets run evaluate the best model again on the test set.


In [ ]:
trainer.evaluate()

{'eval_loss': 0.4502035677433014,
 'eval_f1': 79.072,
 'eval_gen_len': 2.0,
 'eval_runtime': 29.1585,
 'eval_samples_per_second': 10.666,
 'eval_steps_per_second': 1.338,
 'epoch': 12.0}

## 4. Run Inference and Classification Report


In [ ]:
from tqdm.auto import tqdm
results_dict = {'Speaker': [], 'Sentence_Number': [], 'Sentence': [], 'Event': [], 'Target_Event': [], 'Bel(A)': [], 'Bel(B)': [], 'PCG(A)': [], 'PCG(B)': [], 'CG(A)': [], 'CG(B)': [],}

samples_number = len(dataset['test'])
progress_bar = tqdm(range(samples_number))
predictions_list = []
labels_list = []
for i in range(samples_number):
  text = f"Bel(A)={dataset['test']['Bel(A)'][i]},Bel(B)={dataset['test']['Bel(B)'][i]}\n\nInput Event with Context:\n{dataset['test']['Event'][i]}"
  inputs = tokenizer.encode_plus(text, padding='max_length', max_length=512, return_tensors='pt').to('cuda')
  outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=150, num_beams=4, early_stopping=True)
  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
  predictions_list.append(prediction)
  labels_list.append(dataset['test']['CG(A)'][i])

  results_dict['Speaker'].append(dataset['test']['Speaker'][i])
  results_dict['Sentence_Number'].append(dataset['test']['Sentence_Number'][i])
  results_dict['Sentence'].append(text)
  results_dict['Event'].append(dataset['test']['Event'][i])
  results_dict['Target_Event'].append(dataset['test']['Target_Event'][i])
  results_dict['Bel(A)'].append(dataset['test']['Bel(A)'][i])
  results_dict['Bel(B)'].append(dataset['test']['Bel(B)'][i])
  results_dict['PCG(A)'].append(prediction)
  results_dict['PCG(B)'].append(prediction)
  results_dict['CG(A)'].append(dataset['test']['CG(A)'][i])
  results_dict['CG(B)'].append(dataset['test']['CG(B)'][i])

  progress_bar.update(1)

  0%|          | 0/311 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(labels_list, predictions_list, zero_division=0)
print(report)

              precision    recall  f1-score   support

           1       0.94      0.95      0.95       245
           2       0.54      0.50      0.52        30
           3       0.97      1.00      0.99        36

    accuracy                           0.91       311
   macro avg       0.82      0.82      0.82       311
weighted avg       0.91      0.91      0.91       311



In [ ]:
results_df = pd.DataFrame.from_dict(results_dict)

In [ ]:
results_df

,Speaker,Sentence_Number,Sentence,Event,Target_Event,Bel(A),Bel(B),PCG(A),PCG(B),CG(A),CG(B)
0,B,1,"Bel(A)=1,Bel(B)=1\n\nInput Event with Context:...",Previous Sentences: \nTarget Sentence: B took ...,B took the kids to Jill's,1,1,1,1,1,1
1,B,1,"Bel(A)=1,Bel(B)=1\n\nInput Event with Context:...",Previous Sentences: B took the kids to Jill's ...,The kids spent two days at Jill's,1,1,1,1,1,1
2,B,1,"Bel(A)=1,Bel(B)=1\n\nInput Event with Context:...",Previous Sentences: B took the kids to Jill's ...,B guesses Jill couldn't take the kids,1,1,1,1,1,1
3,B,1,"Bel(A)=3,Bel(B)=3\n\nInput Event with Context:...",Previous Sentences: B took the kids to Jill's ...,Jill couldn't take the kids,3,3,1,1,1,1
4,B,1,"Bel(A)=1,Bel(B)=1\n\nInput Event with Context:...",Previous Sentences: B took the kids to Jill's ...,The kid's mom and dad came,1,1,2,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...
306,A,144,"Bel(A)=3,Bel(B)=3\n\nInput Event with Context:...",Previous Sentences: And A hung up the phone It...,A's sons never treat one another like A and B'...,3,3,1,1,1,1
307,B,145,"Bel(A)=1,Bel(B)=1\n\nInput Event with Context:...",Previous Sentences: It's good for A that A hun...,B doesn't think B's kids will be like A and B'...,1,1,1,1,1,1
308,B,145,"Bel(A)=3,Bel(B)=3\n\nInput Event with Context:...",Previous Sentences: It's good for A that A hun...,B's kids will be like A and B's mom and dad,3,3,1,1,1,1
309,A,146,"Bel(A)=1,Bel(B)=1\n\nInput Event with Context:...",Previous Sentences: A asks B why A and B's lif...,A and B's dad just looked at A,1,1,1,1,1,1


In [ ]:
results_df.to_csv('results.csv')

### Using Bel(A) trained model for test on Bel(B)

we should remove Bel(B)=0 for calculate correct classification reports

In [ ]:
test_df = dataset['test'].to_pandas()
test_df = test_df[test_df['CG(B)'] != '0']
dataset['test'] = Dataset.from_pandas(test_df)
dataset['test'] = dataset['test'].remove_columns('__index_level_0__')

In [ ]:
from tqdm.auto import tqdm

samples_number = len(dataset['test'])
progress_bar = tqdm(range(samples_number))
predictions_list = []
labels_list = []
for i in range(samples_number):
  text = f"Bel(A)={dataset['test']['Bel(A)'][i]},Bel(B)={dataset['test']['Bel(B)'][i]}\n\nInput Event with Context:\n{dataset['test']['Event'][i]}"
  inputs = tokenizer.encode_plus(text, padding='max_length', max_length=512, return_tensors='pt').to('cuda')
  outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=150, num_beams=4, early_stopping=True)
  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
  predictions_list.append(prediction)
  labels_list.append(dataset['test']['CG(B)'][i])

  progress_bar.update(1)

  0%|          | 0/310 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(labels_list, predictions_list, zero_division=0)
print(report)

              precision    recall  f1-score   support

           1       0.94      0.95      0.95       245
           2       0.52      0.50      0.51        28
           3       1.00      1.00      1.00        37

    accuracy                           0.91       310
   macro avg       0.82      0.82      0.82       310
weighted avg       0.91      0.91      0.91       310



### Show FP FN

In [ ]:
for i in range(len(predictions_list)):
  if predictions_list[i] == labels_list[i]: print(predictions_list[i], labels_list[i])
  else: print(predictions_list[i], labels_list[i], "Incorrect")


1 1
1 1
1 1
4 4
1 1
4 1 Incorrect
1 1
1 1
1 1
3 1 Incorrect
1 1
4 3 Incorrect
1 1
2 1 Incorrect
1 1
1 1
1 1
3 1 Incorrect
1 1
1 1
1 1
3 1 Incorrect
3 1 Incorrect
1 1
1 1
1 3 Incorrect
2 2
1 1
1 1
1 1
4 4
1 1
1 1
1 1
1 1
1 1
1 1
3 3
1 1
1 1
1 1
1 1
1 1
1 1
1 1
4 4
2 1 Incorrect
1 1
1 1
1 1
1 1
1 1
2 3 Incorrect
4 1 Incorrect
3 1 Incorrect
3 3
1 1
3 3
1 1
1 1
3 4 Incorrect
4 4
1 1
1 1
1 1
2 1 Incorrect
1 1
1 1
1 1
1 1
1 1
1 1
1 2 Incorrect
1 1
1 1
1 1
1 1
1 1
4 4
4 1 Incorrect
1 1
1 1
1 1
4 1 Incorrect
1 1
1 1
4 1 Incorrect
2 2
2 2
1 1
1 1
1 4 Incorrect
1 1
1 1
3 2 Incorrect
4 3 Incorrect
1 1
1 1
2 2
1 2 Incorrect
1 1
3 1 Incorrect
1 1
1 1
1 1
1 1
1 1
1 4 Incorrect
1 2 Incorrect
1 1
1 3 Incorrect
1 1
3 2 Incorrect
1 1
1 1
1 1
1 1
2 1 Incorrect
1 1
1 1
1 1
3 3
1 1
1 1
2 1 Incorrect
1 1
4 4
1 1
1 1
1 1
4 1 Incorrect
1 1
1 1
1 1
1 1
1 1
1 1
1 1
3 2 Incorrect
3 1 Incorrect
3 1 Incorrect
1 1
4 1 Incorrect
2 2
1 1
1 1
4 1 Incorrect
1 1
1 1
1 1
4 1 Incorrect
1 4 Incorrect
1 1
1 1
4 1 Incorrect


### Save Model

In [ ]:
save_directory = "/content/drive/MyDrive/Common Ground Docs/Models/FlanT5_Final_Model_CG_Classification_xCGA_xCGB"
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

### load Model and Test pretrained model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModelForSeq2SeqLM.from_pretrained(save_directory)

In [ ]:
model.to('cuda')

In [ ]:
from tqdm.auto import tqdm

samples_number = len(dataset['test'])
progress_bar = tqdm(range(samples_number))
predictions_list = []
labels_list = []
for i in range(samples_number):
  text = f"Bel(A)={dataset['test']['Bel(A)'][i]},Bel(B)={dataset['test']['Bel(B)'][i]}\n\nInput Event with Context:\n{dataset['test']['Event'][i]}"
  inputs = tokenizer.encode_plus(text, padding='max_length', max_length=512, return_tensors='pt').to('cuda')
  outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=150, num_beams=4, early_stopping=True)
  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
  predictions_list.append(prediction)
  labels_list.append(dataset['test']['Bel(B)'][i])

  progress_bar.update(1)

  0%|          | 0/242 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(labels_list, predictions_list, zero_division=0)
print(report)

              precision    recall  f1-score   support

           1       0.93      0.86      0.90       200
           2       0.46      0.43      0.44        14
           3       0.35      0.47      0.40        17
           4       0.30      0.55      0.39        11

    accuracy                           0.80       242
   macro avg       0.51      0.58      0.53       242
weighted avg       0.83      0.80      0.81       242

